In [1]:
import cv2

> OpenCV默认使用的是BGR. BGR和RGB色彩空间的区别在于图片在色彩通道上的排列顺序不同.


### HSV, HSL和YUV
[链接](https://cloud.fynote.com/share/d/AP5zpeKD#2-1-2-2-HSL_15)

#### HSV（HSB）
+ OpenCV用的最多的色彩空间是HSV.
+ Hue: 色相, 即色彩, 如红色, 蓝色. 用角度度量，取值范围为0°～360°，从红色开始按逆时针方向计算，红色为0°，绿色为120°,蓝色为240°(圆盘，一格一格) R -> G -> B
+ Saturation: 饱和度, 表示颜色接近光谱色的程度。**一种颜色，可以看成是某种光谱色与白色混合的结果**。其中光谱色所占的比例愈大，颜色接近光谱色的程度就愈高，颜色的饱和度也就愈高。饱和度高，颜色则深而艳。光谱色的白光成分为0，饱和度达到最高。通常取值范围为0%～100%，值越大，颜色越饱和（越深，越艳丽）。
+ Value: 明度. 明度表示颜色明亮的程度，对于光源色，明度值与发光体的光亮度有关；对于物体色，此值和物体的透射比或反射比有关。通常取值范围为0%（黑）到100%（白）。最下面是黑色

为什么要使用HSV?

方便OpenCV做图像处理，比如根据hue的值（色相）就可以判断背景颜色.

![hsv](hsv.png)
![hue](hue.png)


#### HSL
![HSL和HSV](HSL和HSV.png)
HSL和HSV差不多.HSL在顶部是纯白的, 不管是什么颜色.


Hue: 色相,是一样的
Saturation: 饱和度，注意不等于HSV的饱和度
Lightness: 亮度
![HSL和HSV的区别](HSL和HSV的区别.png)

#### YUV（比如黑白电视只有Y）
YUV，是一种颜色编码方法。常使用在各个视频处理组件中。 YUV在对照片或视频编码时，考虑到人类的感知能力，允许降低色度的带宽。

“Y”表示明亮度（Luminance或Luma），也就是灰阶值，“U”和“V”表示的则是色度（Chrominance或Chroma），作用是描述影像色彩及饱和度，用于指定像素的颜色。

YUV的发明是由于彩色电视与黑白电视的过渡时期。

YUV最大的优点在于只需占用极少的带宽。

4:4:4表示完全取样。
4:2:2表示2:1的水平取样，垂直完全采样。Y取4个，U和V只取2个
4:2:0表示2:1的水平取样，垂直2：1采样。
4:1:1表示4:1的水平取样，垂直完全采样。

### 色彩空间的转换
+ cvtColor(img, colorspace): 颜色转换的关键API

In [2]:
def callback(value):
    pass

cv2.namedWindow('color', cv2.WINDOW_NORMAL)

img = cv2.imread('cat.jpeg')

# 常见的颜色空间转换
colorspaces = [cv2.COLOR_BGR2RGBA,
               cv2.COLOR_BGR2BGRA,
               cv2.COLOR_BGR2GRAY,
               cv2.COLOR_BGR2HSV,
               cv2.COLOR_BGR2YUV]
#  createTrackbar(trackbarname, winname, value, count, onChange) 创建TrackBar控件, value为trackbar的默认值, count为bar的最大值, 最小为0
cv2.createTrackbar('trackbar', 'color', 0, 4, callback)
while True:
    # getTrackbarPos(trackbarname, winname) 获取TrackBar当前值
    index = cv2.getTrackbarPos('trackbar', 'color')

    #颜色空间转换API
    cvt_img = cv2.cvtColor(img, colorspaces[index])

    cv2.imshow('color', cvt_img)
    key = cv2.waitKey(10)
    if key & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

### 重要数据结构Mat
C++的数据结构 -> numpy ndarray
在python中Mat数据对应numpy的ndarray, 使用numpy提供的深浅拷贝方法即可实现Mat的拷贝

In [3]:
import cv2
import numpy as np

img = cv2.imread('cat.jpeg')
# 浅拷贝
img2 = img.view()

# 深拷贝
img3 = img.copy()

img[10:100, 10:100] = [0, 0, 255]
# cv2.imshow('img', img)
# cv2.imshow('img2', img2)
# cv2.imshow('img3', img3)
cv2.imshow('imgs', np.hstack((img, img2, img3)))
cv2.waitKey(0)
cv2.destroyAllWindows()

#shape属性中包括了三个信息
# #高度，长度 和 通道数
print(img.shape)
#图像占用多大空间 #高度 * 长度 * 通道数
print(img.size)
#图像中每个元素的位深
print(img.dtype)

(480, 640, 3)
921600
uint8


### 通道的分离与合并
+ split(mat)分割图像的通道
+ merge((ch1, ch2, ch3)) 融合多个通道

In [10]:
img = np.zeros((480, 640, 3), np.uint8) # 高度与宽度

b, g, r = cv2.split(img)

b[10:100, 10:100] = 255
g[10:100, 10: 100] = 255

img2 = cv2.merge((b,g,r))
cv2.imshow('b&g', np.vstack((b,g,r)))
cv2.imshow('img2', np.hstack((img,img2)))
cv2.waitKey(0)
cv2.destroyAllWindows()

### 绘制图形

+ 利用OpenCV提供的绘制图形API可以轻松在图像上绘制各种图形, 比如直线, 矩形, 圆, 椭圆等图形.

+ line(img, pt1, pt2, color, thickness, lineType, shift) 画直线

    + img: 在哪个图像上画线
    + pt1, pt2: 开始点, 结束点. 指定线的开始与结束位置
    + color: 颜色
    + thickness: 线宽
    + lineType: 线型.线型为-1, 4, 8, 16, 默认为8
    + shift: 坐标缩放比例.
+ rectangle() 参数同上 画矩形

+ circle(img, center, radius, color[, thickness[, lineType[, shift]]]) 中括号内参数表示可选参数. 画圆

+ ellipse(img, 中心点, 长宽的一半, 角度, 从哪个角度开始, 从哪个角度结束,...)

+ polylines(img, pts, isClosed, color[, thickness[, lineType[, shift]]]) 画多边形
+ fillPoly 填充多边形
+ putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]]) 绘制文本
    + text 要绘制的文本
    + org 文本在图片中的左下角坐标
    + fontFace 字体类型即字体
    + fontScale 字体大小

In [ ]:
import cv2
import numpy as np
# 纯黑的背景图
img = np.zeros((640, 800, 3), np.uint8)

# 图片，起始点，结束点，颜色，线条宽度，线条类型（0，4，8，16，越大锯齿越不明显），坐标缩放比例
cv2.line(img, (10, 20), (300, 400), (0, 0, 255), 5, 4)
cv2.line(img, (300, 400), (500, 480), (0, 255, 0), 1, 16)

# 画矩形
# pts：对角上的两个点，thickness=-1，填充
cv2.rectangle(img, (10,10), (100, 100), (0, 0, 255), 0)

# 画圆
# 图像，圆心，半径
cv2.circle(img, (320, 240), 100, (0, 0, 255))
cv2.circle(img, (500, 240), 50, (0, 0, 255), 16, -1)

# 画椭圆
# img, 中心点, 长宽的一半, 角度, 【从哪个角度开始, 从哪个角度结束】,...
# axes-> axis的复数
cv2.ellipse(img, (320, 240), (100, 50), 90, 0, 270, (255, 0, 255), -1)

#画多边形
# img, pts, isClosed是否闭合, color[, thickness[, lineType[, shift]]]
# 多边形的每个点
pts = np.array([(250, 100), (150, 300), (50, 280)], np.int32)
cv2.polylines(img, [pts], False, (0, 255, 255),16)

#填充多边形
cv2.fillPoly(img, [np.array([(450, 200), (250, 400), (10, 280)])], (255, 255, 0))
# text 要绘制的文本  org 文本在图片中的左下角坐标    + fontFace 字体类型即字体    + fontScale 字体大小
cv2.putText(img, "Hello OpenCV!", (20, 550), cv2.FONT_HERSHEY_TRIPLEX, 3, (255,0,0))
cv2.imshow('draw', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [72]:
### 修复中文乱码
from PIL import ImageFont, ImageDraw, Image
img = np.full((200,200,3),fill_value=255, dtype=np.uint8)
# C:\Windows\Fonts复制字体文件，字体大小45
font = ImageFont.truetype('msyh.ttc', 15)
img_pil = Image.fromarray(img)
draw = ImageDraw.Draw(img_pil)
draw.text((10,150), '你好，OpenCV', font=font, fill=(0, 255,0,0))
img = np.array(img_pil)
cv2.imshow('Hello', img)
cv2.waitKey(0)
cv2.destroyAllWindows()